In [45]:
import sys

import pandas as pd
import s3fs

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
raw_data = hd.read_raw_sample("777")
clean = md.clean_data(raw_data).pipe(md.create_vars)
hd.inspect(clean)

Time for read_raw_sample: 4.23 seconds
(590,042, 34)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,desco,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,user_age,entropy_sptac,region
0,688300,2012-02-01,777,400.00,mdbremoved,NaN,transfers,other_transfers,0.0,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-11-13,True,<mdbremoved> - s/o,364.220001,non merchant mbl,transfers,other account,other account,u,201202,-2276.077637,24469.546875,52.0,2.838295,North West
1,688299,2012-02-01,777,3.03,aviva pa,aviva,spend,finance,0.0,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-08-15,True,aviva pa - d/d,364.220001,aviva,health insurance,life insurance,life insurance,u,201202,-2276.077637,24469.546875,52.0,2.838295,North West


Test selector pipeline

In [359]:
selected = sl.select_sample(clean)
selected.head(3)

add_raw_count
255
current_and_savings_account
148
min_number_of_months
136
no_missing_months
113
account_balances_available
49
min_monthly_grocery_transactions
16
income_pmts
16
income_amount
12
max_accounts
11
add_final_count
11


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,desco,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,user_age,entropy_sptac,region
32993,2994252,2012-11-01,14777,12.12,tv licence mbp 6832,tv licensing,spend,communication,0.0,sw11 5,2013-01-07,NaN,1986.0,2013-01-07,247061,2015-01-13 05:19:00,lloyds bank,current,2014-07-18,2018-04-30,True,tv licence mbp xxxxxx6832,729.590027,tv licensing,tv licence,NaN,tv licence,u,201211,-479.021179,14587.92041,35.0,2.649165,London
32994,2994250,2012-11-01,14777,900.00,mdbremoved,NaN,NaN,NaN,0.0,sw11 5,2013-01-07,NaN,1986.0,2013-01-07,247061,2015-01-13 05:19:00,lloyds bank,current,2014-07-18,2015-03-19,True,<mdbremoved>,729.590027,NaN,NaN,rent,rent,u,201211,-479.021179,14587.92041,35.0,2.649165,London
32995,2994127,2012-11-01,14777,25.00,mdbremoved,NaN,NaN,NaN,0.0,sw11 5,2013-01-07,NaN,1986.0,2013-01-07,247060,2015-01-13 05:19:00,lloyds bank,current,2014-07-18,2015-03-19,True,<mdbremoved>,-733.090027,NaN,NaN,unsecured loan repayment,unsecured loan repayment,u,201211,-2113.720703,14587.92041,35.0,2.649165,London


In [361]:
fs.ls('3di-data-mdb/raw')

['3di-data-mdb/raw/',
 '3di-data-mdb/raw/20200630_UserLoginsForNeedham.csv',
 '3di-data-mdb/raw/mdb_000.csv',
 '3di-data-mdb/raw/mdb_000.parquet',
 '3di-data-mdb/raw/mdb_777.csv',
 '3di-data-mdb/raw/mdb_777.parquet',
 '3di-data-mdb/raw/mdb_X77.csv',
 '3di-data-mdb/raw/mdb_X77.parquet',
 '3di-data-mdb/raw/mdb_XX7.csv',
 '3di-data-mdb/raw/mdb_XX7.parquet',
 '3di-data-mdb/raw/mdb_costa.csv']